In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

In [5]:
data = pd.read_json('Rental Prices (Kaggle)/properties.json', lines=True)
rental_prices = pd.DataFrame(data)

### For the rental table, we need to preprocess the table by linking the city to the municipality. This can be done by using a seperate table by CBS.

In [8]:
import requests

base_table_url = "https://opendata.cbs.nl/ODataApi/odata/85210NED/TypedDataSet"

response = requests.get(base_table_url)

if response.status_code == 200:
    base_table_data = response.json()['value']
    woonplaatsen_nl = pd.DataFrame(base_table_data)

else:
    print(f"Failed request")

In [10]:
woonplaatsen_nl.head()

,ID,Woonplaatsen,Woonplaatscode_1,Naam_2,Code_3,Naam_4,Code_5,Naam_6,Code_7
0,0,WP1925,WP1925,Almelo,GM0141,Overijssel,PV23,Oost-Nederland,LD02
1,1,WP2774,WP2774,Veere,GM0717,Zeeland,PV29,West-Nederland,LD03
2,2,WP2145,WP2145,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01
3,3,WP2571,WP2571,Aalsmeer,GM0358,Noord-Holland,PV27,West-Nederland,LD03
4,4,WP1600,WP1600,Haarlemmermeer,GM0394,Noord-Holland,PV27,West-Nederland,LD03


In [22]:
woonplaatsen_nl['Naam_2'] = woonplaatsen_nl['Naam_2'].str.strip()

# only the unique municipalities are relevant
woonplaatsen_nl_update = woonplaatsen_nl.drop(columns = ['ID', 'Woonplaatsen', 'Woonplaatscode_1'])
woonplaatsen_nl_update_unique = woonplaatsen_nl_update.drop_duplicates().reset_index().drop(columns = ['index'])
woonplaatsen_nl_update_unique.head()

,Naam_2,Code_3,Naam_4,Code_5,Naam_6,Code_7
0,Almelo,GM0141,Overijssel,PV23,Oost-Nederland,LD02
1,Veere,GM0717,Zeeland,PV29,West-Nederland,LD03
2,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01
3,Aalsmeer,GM0358,Noord-Holland,PV27,West-Nederland,LD03
4,Haarlemmermeer,GM0394,Noord-Holland,PV27,West-Nederland,LD03


In [66]:
merged_rental_table_and_municipality = pd.merge(rental_prices, woonplaatsen_nl_update_unique, left_on=['city'], right_on=['Naam_2'], how='inner')
merged_rental_table_and_municipality.head(3)

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription,Naam_2,Code_3,Naam_4,Code_5,Naam_6,Code_7
0,{'$oid': '5d2b113a43cbfd7c77a998f4'},room-1686123,14 m2,14,Rotterdam,https://resources.kamernet.nl/image/913b4b03-5...,done,{'$date': '2019-07-26T22:18:23.018+0000'},"[{'$date': '2019-07-14T11:25:46.511+0000'}, {'...",{'$date': '2019-07-14T11:25:46.511+0000'},Unfurnished,{'$date': '2019-07-26T22:18:23.142+0000'},51.896601,4.514993,3074HN,4w,Room,26-06-'19 - Indefinite period,500,,"€ 500,-",kamernet,West-Varkenoordseweg,https://kamernet.nl/en/for-rent/room-rotterdam...,50.0,\n € 50\n ...,500.0,\n € 500\n ...,"Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...","Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...",{'$date': '2019-07-22T07:10:41.849+0000'},Unknown,Mixed,Yes,true,Shared,None,16 years -\n 99 years,16 years -\n 99 years,1 person,Not important,Not important,Not important,Not important,Not important,"Room for rent in Rotterdam, West-Varkenoordse...",Room for rent in Rotterdam €500 | Kamernet,No,0,\n € 0\n ...,5,Shared,No,Shared,Huize west,4680711.0,21-07-2019,26-06-2019,https://resources.kamernet.nl/Content/images/s...,NaN,Rotterdam,GM0599,Zuid-Holland,PV28,West-Nederland,LD03
1,{'$oid': '5d2b113a43cbfd7c77a9991a'},studio-1691193,30 m2,30,Amsterdam,https://resources.kamernet.nl/image/5e11d6b5-8...,done,{'$date': '2019-08-10T22:28:46.099+0000'},"[{'$date': '2019-07-14T11:25:46.677+0000'}, {'...",{'$date': '2019-07-14T11:25:46.677+0000'},Furnished,{'$date': '2019-08-10T22:28:46.229+0000'},52.370200,4.920721,1018AS,4w,Studio,15-08-'19 - Indefinite period,950,Utilities incl.,"€ 950,- Utilities incl.",kamernet,Parelstraat,https://kamernet.nl/en/for-rent/studio-amsterd...,0.0,\n € 0\n ...,895.0,\n € 895\n ...,"Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...","Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...",{'$date': '2019-07-22T06:29:33.112+0000'},Unknown,Unknown,Yes,true,Own,Own,18 years -\n 99 years,18 years -\n 99 years,1 person,Not important,Not important,Not important,"Working student, Working","Working student, Working","Studio for rent in Amsterdam, Parelstraat, fo...",Studio for rent in Amsterdam €950 | Kamernet,No,0,\n € 0\n ...,None,Own,No,Own,Cor,1865530.0,20-07-2019,05-01-2012,https://resources.kamernet.nl/Content/images/p...,NaN,Amsterdam,GM0363,Noord-Holland,PV27,West-Nederland,LD03
2,{'$oid': '5d2b113a43cbfd7c77a99931'},room-1690545,11 m2,11,Amsterdam,https://resources.kamernet.nl/image/74b93a27-a...,done,{'$date': '2019-10-02T22:00:33.141+0000'},"[{'$date': '2019-07-14T11:25:46.834+0000'}, {'...",{'$date': '2019-07-14T11:25:46.834+0000'},Furnished,{'$date': '2019-10-02T22:00:33.264+0000'},52.350880,4.854786,1075SB,09 Jul,Room,01-08-'19 - Indefinite period,1000,Utilities incl.,"€ 1000,- Utilities incl.",kamernet,Zeilstraat,https://kamernet.nl/en/for-rent/room-amsterdam...,NaN,\n -\n ...,1000.0,\n € 1000\n ...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e..

In [70]:
dates = merged_rental_table_and_municipality['firstSeenAt'].apply(lambda x: x['$date'] if isinstance(x, dict) and '$date' in x else None)
dates = pd.to_datetime(dates) 
merged_rental_table_and_municipality['firstSeenYear'] = dates.dt.year 
merged_rental_table_and_municipality.head(2)

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription,Naam_2,Code_3,Naam_4,Code_5,Naam_6,Code_7,firstSeenYear
0,{'$oid': '5d2b113a43cbfd7c77a998f4'},room-1686123,14 m2,14,Rotterdam,https://resources.kamernet.nl/image/913b4b03-5...,done,{'$date': '2019-07-26T22:18:23.018+0000'},"[{'$date': '2019-07-14T11:25:46.511+0000'}, {'...",{'$date': '2019-07-14T11:25:46.511+0000'},Unfurnished,{'$date': '2019-07-26T22:18:23.142+0000'},51.896601,4.514993,3074HN,4w,Room,26-06-'19 - Indefinite period,500,,"€ 500,-",kamernet,West-Varkenoordseweg,https://kamernet.nl/en/for-rent/room-rotterdam...,50.0,\n € 50\n ...,500.0,\n € 500\n ...,"Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...","Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...",{'$date': '2019-07-22T07:10:41.849+0000'},Unknown,Mixed,Yes,true,Shared,None,16 years -\n 99 years,16 years -\n 99 years,1 person,Not important,Not important,Not important,Not important,Not important,"Room for rent in Rotterdam, West-Varkenoordse...",Room for rent in Rotterdam €500 | Kamernet,No,0,\n € 0\n ...,5,Shared,No,Shared,Huize west,4680711.0,21-07-2019,26-06-2019,https://resources.kamernet.nl/Content/images/s...,NaN,Rotterdam,GM0599,Zuid-Holland,PV28,West-Nederland,LD03,2019
1,{'$oid': '5d2b113a43cbfd7c77a9991a'},studio-1691193,30 m2,30,Amsterdam,https://resources.kamernet.nl/image/5e11d6b5-8...,done,{'$date': '2019-08-10T22:28:46.099+0000'},"[{'$date': '2019-07-14T11:25:46.677+0000'}, {'...",{'$date': '2019-07-14T11:25:46.677+0000'},Furnished,{'$date': '2019-08-10T22:28:46.229+0000'},52.370200,4.920721,1018AS,4w,Studio,15-08-'19 - Indefinite period,950,Utilities incl.,"€ 950,- Utilities incl.",kamernet,Parelstraat,https://kamernet.nl/en/for-rent/studio-amsterd...,0.0,\n € 0\n ...,895.0,\n € 895\n ...,"Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...","Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...",{'$date': '2019-07-22T06:29:33.112+0000'},Unknown,Unknown,Yes,true,Own,Own,18 years -\n 99 years,18 years -\n 99 years,1 person,Not important,Not important,Not important,"Working student, Working","Working student, Working","Studio for rent in Amsterdam, Parelstraat, fo...",Studio for rent in Amsterdam €950 | Kamernet,No,0,\n € 0\n ...,None,Own,No,Own,Cor,1865530.0,20-07-2019,05-01-2012,https://resources.kamernet.nl/Content/images/p...,NaN,Amsterdam,GM0363,Noord-Holland,PV27,West-Nederland,LD03,2019


In [72]:
data_2019 = merged_rental_table_and_municipality[merged_rental_table_and_municipality['firstSeenYear'] == 2019]

average_price_per_gemeente = (
    data_2019.groupby(['Naam_2', 'Code_3'])['rent']
    .mean()
    .reset_index()
)

average_price_per_gemeente.columns = ['Gemeente', 'Gemeentecode', 'Average_Room_Price']
average_price_per_gemeente

,Gemeente,Gemeentecode,Average_Room_Price
0,Aalsmeer,GM0358,1090.529412
1,Alblasserdam,GM0482,685.000000
2,Alkmaar,GM0361,650.187970
3,Almelo,GM0141,434.942857
4,Almere,GM0034,671.753555
5,Alphen aan den Rijn,GM0484,574.761905
6,Amersfoort,GM0307,602.111111
7,Amstelveen,GM0362,985.863208
8,Amsterdam,GM0363,969.230279
9,Apeldoorn,GM0200,619.136364


In [78]:
average_price_per_gemeente.to_csv('average_rental_price_per_gemeente.csv', index=False)